In [2]:
import pandas as pd
import re

In [2]:
df = pd.read_excel('/content/drive/MyDrive/대한항공 전처리 전.xlsx')

In [3]:
df = df.drop(['Unnamed: 0'],axis = 1)

In [ ]:
df

In [8]:
df = df[['predict_date','article_text','대한타겟']]

In [9]:
def clean_text(inputString):
    text_rmv = re.sub('[+]','증가', inputString)
    text_rmv = re.sub('[-]','감소', text_rmv)
    text_rmv = re.sub('[↑▲]','상승', text_rmv)
    text_rmv = re.sub('[↓△]','하락', text_rmv)

    return text_rmv

In [10]:
df['article_text'] = df['article_text'].apply(clean_text)

In [11]:
def preprocess_text(text):
    patterns = [
        r'\.([^\.]*(?:기자|특파원|교수|작가|대표|논설|고문|주필|부문장|팀장|장관|원장|연구원|이사장|위원|실장|차장|부장|에세이|화백|사설|소장|단장|과장|기획자|큐레이터|저작권|평론가|©|©|ⓒ|\@|\/|=|▶|무단|전재|재배포|금지|\[|\]|\(\))[^\.]*)$',
        r'원본보기',
        r'아이콘',
        r'AD',
        r'자료',
        r'ADVERTISEMENT',
        r'Advertisement',
        r'viewer',
        r'김기혁',
        r'조영석',
        r'나종덕',
        r'김홍재',
        r'느낌표',
        r'박세웅',
        r'이새롬',
    ]

    for pattern in patterns:
        regex = re.compile(pattern)
        text = re.sub(regex, '', text)

    text = re.sub('[^\w.,%]', '', text)
    text = ' '.join(text.split())
    text = re.sub(r"\S+@\S+", "", text)  # 이메일 패턴 제거
    text = re.sub(r"[가-힣]+기자", "", text)  # 기자 이름 패턴 제거 (한글 기자 이름)
    text = re.sub(r'\b\w+\s?기자\b', '', text)
    text = re.sub(r'\b\w+@\w+\.\w+\b', '', text)
    text = re.sub(r'\b\w+\.com\b', '', text)
    text = re.sub(r'[^.!?]+$', '', text)

    return text

In [12]:
df['article_text'] = df['article_text'].apply(preprocess_text)

In [14]:
df

,predict_date,article_text,대한타겟
0,2022-01-03,상승26일오후경기도의정부체육관에서열린2021의정부도드람컵프로배구대회순위결정전에서흥국...,0
1,2022-01-04,상승사천공항경남도는진에어가오는28일부터김포감소사천진주노선에신규취항한다고4일밝혔다.2...,0
2,2022-01-05,인천중구영종국제도시대한항공항공정비단지부지.인천시제공대한항공이인천영종도에항공정비MRO...,0
3,2022-01-06,"인천시감소대한항공,항공정비MRO산업발전업무협약체결대한항공,3346억투자2025년완공...",0
4,2022-01-07,6일프로탁구리그총괄위원장이된한국탁구의레전드안재형전탁구대표팀감독은리그를활성화시켜한국탁...,1
...,...,...,...
478,2023-05-26,파이낸셜뉴스항공사들이연이어국제항공운송협회IATA리튬배터리항공운송인증CEIVLI감소B...,0
479,2023-05-28,속보코로나19신규확진1만7796명1주전과비슷27일전국에서1만7000명대의코로나19신...,1
480,2023-05-29,슈퍼태풍마와르피해로운영이중단된괌국제공항이오는29일부터운영이재개된다.외교부는29일월요...,1
481,2023-05-30,파이낸셜뉴스사측과임금인상률합의를하지못한아시아나항공조종사노동조합이쟁의행위에돌입한다.최...,1


In [13]:
pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 46.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 77.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 56.9 MB/s eta 0:00:00


In [15]:
import torch
from transformers import AutoTokenizer, AutoModel

In [16]:
model_name = "beomi/kcbert-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)


Some weights of the model checkpoint at beomi/kcbert-base were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [17]:
def encode_text(text):
    sentences = text.split(".")  # 문장을 "."을 기준으로 분리
    encoded_sentences = []

    for sentence in sentences:
        marked_sentence = "[CLS] " + sentence.strip() + " [SEP]"  # [CLS]와 [SEP] 토큰 추가

        tokens = tokenizer.encode(marked_sentence, add_special_tokens=True)
        tokens_tensor = torch.tensor(tokens).unsqueeze(0)

        with torch.no_grad():
            outputs = model(tokens_tensor)
            embeddings = outputs[0].squeeze(0)

        encoded_sentences.append(embeddings)

    return encoded_sentences

In [18]:

df["encoded_sentences"] = None


In [ ]:
for i, text in enumerate(df["article_text"]):
    embeddings = encode_text(text)
    df.at[i, "encoded_sentences"] = embeddings

In [ ]:
df.to_excel('/content/drive/MyDrive/Colab Notebooks/대한항공 임베딩 완.xlsx')

In [ ]:
df.head()

In [18]:
df = df.drop(0)